<a href="https://colab.research.google.com/github/ramansandhu17/Airbnb_Analysis/blob/main/labeled_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df=pd.read_csv('cleaned_reviews.csv')

df.head()

,Date,Occupancy,PricePerNight,Revenue,PropertyID,GuestID,ReviewSummary,CleanedReview
0,8/5/24,1,169,169,P001,G1001,"A perfect stay, felt like home!",perfect stay felt like home
1,8/6/24,1,123,123,P001,G1001,"Loved the location, very convenient!",loved location convenient
2,8/7/24,1,85,85,P001,G1001,"A perfect stay, felt like home!",perfect stay felt like home
3,8/8/24,1,134,134,P001,G1001,"Super cozy, highly recommend!",super cozy highly recommend
4,8/9/24,0,196,0,P001,G0,NaN,NaN


In [6]:
#Lets read the property rules

pdf = pd.read_csv('properties.csv', encoding='ISO-8859-1')

pdf.head()

,PropertyID,Name,Bedrooms,Bathrooms,Location,State,PropertyRules
0,P001,Private 1 bedroom suite,1,1.0,Mukilteo,WA,"This is a non-smoking property, and no pets ar..."
1,P002,Beachfront Villa,3,2.0,Edmonds,WA,We kindly ask that guests respect the natural ...
2,P003,City Center Loft,2,1.5,Seattle,WA,This loft is located in a lively city center. ...
3,P004,Cozy Cabin,1,1.0,NorthBend,WA,"This is a peaceful cabin in the woods, and we ..."
4,P005,Modern Studio,0,1.0,Bellevue,WA,This is a non-smoking and pet-free studio. Ple...


In [9]:
#Convert the rules from a string to a list of individual rules

pdf['rules_list']=pdf['PropertyRules'].str.split(',')

pdf['rules_list']=pdf['rules_list'].apply(lambda x:[r.strip() for r in x])

rules_dict=dict(zip(pdf['PropertyID'],pdf['rules_list']))

for prop_id,rules in rules_dict.items():
  print(f"{prop_id}:{rules}")

P001:['This is a non-smoking property', 'and no pets are allowed. Please be mindful of your noise levels', 'especially after 9 PM', 'as we want to maintain a peaceful environment for all guests. Maximum occupancy is 2 guests only. Any parties or gatherings are strictly prohibited. Please respect the neighborsÕ privacy and avoid any disruptive behavior. We have security cameras on the exterior of the property for your safety and to monitor the entrance. No outside guests are allowed unless pre-approved. Please use the designated parking spot only.']
P002:['We kindly ask that guests respect the natural beauty of the beach. No smoking at all times. No pets are allowed on the property', 'and any loud parties or gatherings are not permitted. Guests must clean off any sand before entering the villa to avoid damage to the flooring. We ask that you respect the privacy of the neighbors at all times. The property has a private parking area for up to 2 vehicles. Maximum occupancy is 4 guests. For

In [13]:
#Check if the violation happened by matching words from pdf to df

def check_violation(review,rules):
  if not isinstance(review,str):
    return 0

  review=review.lower()
  for rule in rules:
    rule_words=rule.lower().split()

    #checking for all words without no/nans in review with any
    if all(word in review for word in rule_words):
      return 1
  return 0

In [19]:
#Apply the function to each row in the reviews

df['violating'] = df.apply(
    lambda row: check_violation(row['ReviewSummary'], rules_dict.get(row['PropertyID'], [])),
    axis=1
)

df.value_counts('violating')

,count
violating,
0,904
1,1


In [20]:
df[df['violating']==1][['PropertyID','CleanedReview']]

,PropertyID,CleanedReview
385,P003,really surprised host brought concern pet pers...
